In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import warnings
import keras
import keras.backend as K
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
from keras.models import Sequential
import tensorflow as tf
import gc
from numba import jit
from IPython.display import display, clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sns.set_style("whitegrid")

In [5]:
df = pd.read_csv(r'datasetv6.csv',sep=',')
df['Y'] = df['Y'].astype(int)
#df=df[df['Y'] != 5]
#df=df.drop(range(60))
#df.drop(columns=['Participant'], axis=1, inplace=True)
#df.drop(columns=['Circularity', 'EAR_N','MAR_N','Circularity_N', 'MOE_N'], axis=1, inplace=True)

print(df.shape)
print(df)

(10000, 4)
           EAR       MAR       MOE  Y
0     0.352255  2.098859  5.958359  0
1     0.324341  1.695003  5.225990  0
2     0.367728  1.611210  4.381528  0
3     0.326678  1.506216  4.610711  0
4     0.204691  1.478582  7.223492  0
...        ...       ...       ... ..
9995  0.283778  1.399390  4.931281  1
9996  0.240754  1.402620  5.825960  1
9997  0.254672  1.410812  5.539728  1
9998  0.250514  1.404537  5.606619  1
9999  0.244891  1.459801  5.961024  1

[10000 rows x 4 columns]


In [6]:
train_percentage = 0.60
train_index = int(len(df)*train_percentage)
test_index = len(df)-train_index
     

In [7]:
df_train = df[:train_index]
df_test = df[-test_index:]
print(df_train)
print(df_test)

           EAR       MAR       MOE  Y
0     0.352255  2.098859  5.958359  0
1     0.324341  1.695003  5.225990  0
2     0.367728  1.611210  4.381528  0
3     0.326678  1.506216  4.610711  0
4     0.204691  1.478582  7.223492  0
...        ...       ...       ... ..
5995  0.284498  1.366812  4.804291  0
5996  0.250000  1.297942  5.191769  0
5997  0.269122  1.343438  4.991936  0
5998  0.274734  1.380538  5.024992  0
5999  0.270142  1.340271  4.961351  0

[6000 rows x 4 columns]
           EAR       MAR       MOE  Y
6000  0.273908  1.207233  4.407438  1
6001  0.303636  1.292833  4.257841  1
6002  0.256382  1.181273  4.607475  1
6003  0.288730  1.183716  4.099735  1
6004  0.281647  1.186893  4.214119  1
...        ...       ...       ... ..
9995  0.283778  1.399390  4.931281  1
9996  0.240754  1.402620  5.825960  1
9997  0.254672  1.410812  5.539728  1
9998  0.250514  1.404537  5.606619  1
9999  0.244891  1.459801  5.961024  1

[4000 rows x 4 columns]


In [9]:


x_test = df_test.drop(["Y"],axis=1)
y_test = df_test["Y"]


print(x_test)
print(y_test)

           EAR       MAR       MOE
6000  0.273908  1.207233  4.407438
6001  0.303636  1.292833  4.257841
6002  0.256382  1.181273  4.607475
6003  0.288730  1.183716  4.099735
6004  0.281647  1.186893  4.214119
...        ...       ...       ...
9995  0.283778  1.399390  4.931281
9996  0.240754  1.402620  5.825960
9997  0.254672  1.410812  5.539728
9998  0.250514  1.404537  5.606619
9999  0.244891  1.459801  5.961024

[4000 rows x 3 columns]
6000    1
6001    1
6002    1
6003    1
6004    1
       ..
9995    1
9996    1
9997    1
9998    1
9999    1
Name: Y, Length: 4000, dtype: int32


In [10]:
x_train = df_train.drop('Y',axis=1)
y_train = df_train['Y']

print(x_train)
print(y_train)

           EAR       MAR       MOE
0     0.352255  2.098859  5.958359
1     0.324341  1.695003  5.225990
2     0.367728  1.611210  4.381528
3     0.326678  1.506216  4.610711
4     0.204691  1.478582  7.223492
...        ...       ...       ...
5995  0.284498  1.366812  4.804291
5996  0.250000  1.297942  5.191769
5997  0.269122  1.343438  4.991936
5998  0.274734  1.380538  5.024992
5999  0.270142  1.340271  4.961351

[6000 rows x 3 columns]
0       0
1       0
2       0
3       0
4       0
       ..
5995    0
5996    0
5997    0
5998    0
5999    0
Name: Y, Length: 6000, dtype: int32


In [11]:
print(x_test.shape)
print(x_train.shape)

(4000, 3)
(6000, 3)


In [12]:
print(y_test.shape)
print(y_train.shape)

(4000,)
(6000,)


In [14]:
import numpy as np
x_shaped_train  = np.array(x_train).reshape(1200,5,3) #6000 : 5
x_shaped_test  = np.array(x_test).reshape(800,5,3) #4000 :5

print(x_shaped_train)

[[[0.35225453 2.09885904 5.95835932]
  [0.32434097 1.69500266 5.22599   ]
  [0.36772773 1.6112095  4.38152845]
  [0.32667754 1.50621557 4.61071055]
  [0.20469072 1.47858173 7.22349172]]

 [[0.32224899 1.55498116 4.82540281]
  [0.34821984 1.55415176 4.46313385]
  [0.2845396  1.63067506 5.73092473]
  [0.29654214 1.4957652  5.04402242]
  [0.35158154 1.44347702 4.10566788]]

 [[0.33352509 1.51293017 4.53618103]
  [0.33282012 1.59081263 4.77979709]
  [0.33790474 1.82657243 5.40558398]
  [0.34053958 1.75960239 5.16710092]
  [0.28408014 1.69294264 5.95938393]]

 ...

 [[0.23368903 1.37334475 5.87680444]
  [0.26169749 1.33673337 5.10793345]
  [0.26169749 1.33871518 5.11550637]
  [0.25080049 1.36569831 5.44535747]
  [0.25080049 1.41148219 5.62790847]]

 [[0.25643643 1.35128692 5.26948107]
  [0.24699346 1.33695854 5.41293088]
  [0.2502103  1.32250995 5.28559345]
  [0.27638835 1.38137968 4.99796635]
  [0.26074123 1.36826654 5.2476033 ]]

 [[0.28449807 1.36681153 4.80429106]
  [0.25       1.297942

In [15]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_shaped_train  = []
for i in range(0, len(y_train), 5):
  y_shaped_train.append([y_train[i]])
print(len(y_shaped_train))

y_shaped_test  = []
for i in range(0, len(y_test), 5):
  y_shaped_test.append([y_test[i]])
print(len(y_shaped_test))

1200
800


In [16]:
print(x_shaped_train.shape)
print(x_shaped_test.shape)

(1200, 5, 3)
(800, 5, 3)


In [17]:
y_shaped_train = np.array(y_shaped_train)
print(y_shaped_train.shape)

y_shaped_test = np.array(y_shaped_test)
print(y_shaped_test.shape)

(1200, 1)
(800, 1)


In [18]:
def keras_auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

CLSTM

In [20]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
# from keras.layers.recurrent import LSTM
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
# from keras.layers.wrappers import TimeDistributed
from tensorflow.keras.layers import Flatten, GRU, Dropout, TimeDistributed, Dense
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys
import tensorflow as tf

#definisi model
metrics = ['accuracy']
model = Sequential()

model.add(TimeDistributed(Conv1D(64, kernel_size = 3, activation = 'relu', input_shape = (5,3))))
# model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation = 'relu'))
model.add(TimeDistributed(Flatten()))

# model.add(Dense(1024, activation='sigmoid'))
model.add(LSTM(300, return_sequences=True,
                       input_shape=(5, 3,),
                       dropout=0.5))

model.add(Dense(100, activation='relu'))
model.add(Dense(1,activation = 'sigmoid'))

optimizer = Adam(learning_rate=0.00001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# patient early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

#model checkpoint
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

#fit model
history=model.fit(tf.expand_dims(x_shaped_train, axis=-1), y_shaped_train, validation_data = (x_shaped_test,y_shaped_test), epochs=500,  verbose=0, callbacks=[es,mc])#batch_size= 50

# load the saved model
saved_model = load_model('best_model.h5')

#evaluate the model
_, train_acc = model.evaluate(x_shaped_train, y_shaped_train, verbose=0)
_, test_acc = model.evaluate(x_shaped_test,y_shaped_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# #plot training history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()
     


Epoch 1: val_accuracy improved from -inf to 0.45125, saving model to best_model.h5

Epoch 2: val_accuracy improved from 0.45125 to 0.54000, saving model to best_model.h5

Epoch 3: val_accuracy improved from 0.54000 to 0.55000, saving model to best_model.h5

Epoch 4: val_accuracy did not improve from 0.55000

Epoch 5: val_accuracy did not improve from 0.55000

Epoch 6: val_accuracy did not improve from 0.55000

Epoch 7: val_accuracy did not improve from 0.55000

Epoch 8: val_accuracy did not improve from 0.55000

Epoch 9: val_accuracy did not improve from 0.55000

Epoch 10: val_accuracy did not improve from 0.55000

Epoch 11: val_accuracy did not improve from 0.55000

Epoch 12: val_accuracy did not improve from 0.55000

Epoch 13: val_accuracy did not improve from 0.55000

Epoch 14: val_accuracy did not improve from 0.55000

Epoch 15: val_accuracy did not improve from 0.55000

Epoch 16: val_accuracy did not improve from 0.55000

Epoch 17: val_accuracy did not improve from 0.55000

Epoch

LSTM

In [26]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
#from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from collections import deque
import sys

metrics = ['accuracy']
model = Sequential()
model.add(Dense(1024, activation='sigmoid'))
model.add(LSTM(512, return_sequences=True,
                       input_shape=(5, 4,),
                       dropout=0.5))

model.add(Flatten())
# Dense is fully connected layer. 16 hidden units
# activation for lstm is basically sigmoid or tanh
model.add(Dense(216, activation='sigmoid')) #FC1
model.add(Dense(32, activation='tanh')) #FC2
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))#FC3
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh'))#Output Layer
optimizer = Adam(lr=0.00005)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics)

# patient early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

#model checkpoint
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

#fit model
history=model.fit(x_shaped_train, y_shaped_train, validation_data = (x_shaped_test,y_shaped_test), epochs=500, batch_size=  32)

# load the saved model
saved_model = load_model('best_model.h5')

#evaluate the model
_, train_acc = model.evaluate(x_shaped_train, y_shaped_train, verbose=0)
_, test_acc = model.evaluate(x_shaped_test,y_shaped_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

#model.fit(x_shaped_train, y_shaped_train, validation_data = (x_shaped_test,y_shaped_test), epochs=500, batch_size=  32)
#model.predict_classes(x_shaped_test)
#predictions = model.predict(x_shaped_test)
#predicted_classes = np.argmax(predictions, axis=1)

Epoch 1/500
38/38 [==============================] - 7s 142ms/step - loss: 3.2840 - accuracy: 0.4967 - val_loss: 0.6964 - val_accuracy: 0.5500
Epoch 2/500
38/38 [==============================] - 7s 189ms/step - loss: 2.5658 - accuracy: 0.4875 - val_loss: 0.7816 - val_accuracy: 0.5500
Epoch 3/500
38/38 [==============================] - 7s 197ms/step - loss: 2.3233 - accuracy: 0.4842 - val_loss: 0.7213 - val_accuracy: 0.5500
Epoch 4/500
38/38 [==============================] - 7s 176ms/step - loss: 2.1770 - accuracy: 0.5042 - val_loss: 0.7120 - val_accuracy: 0.5500
Epoch 5/500
38/38 [==============================] - 6s 154ms/step - loss: 1.8930 - accuracy: 0.5075 - val_loss: 0.7301 - val_accuracy: 0.5500
Epoch 6/500
38/38 [==============================] - 6s 157ms/step - loss: 1.7392 - accuracy: 0.4917 - val_loss: 0.6883 - val_accuracy: 0.5500
Epoch 7/500
38/38 [==============================] - 6s 166ms/step - loss: 1.5959 - accuracy: 0.5025 - val_loss: 0.6952 - val_accuracy: 0.4500